In [1]:
import os
import numpy as np
import librosa

def extract_features(file_path, max_length=1000):
    audio, sr = librosa.load(file_path, mono=True)
    # 1. Coeficientes MFCC
    mfccs = librosa.feature.mfcc(y=audio, sr=sr)
    mfcc_features = np.concatenate((mfccs.mean(axis=1), mfccs.std(axis=1)))
    # 2. Cromagrama
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    # 3. Contraste espectral
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    # 4. Tonnetz
    tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
    # 5. Tempograma
    tempo, tempogram = librosa.beat.beat_track(y=audio, sr=sr)
    # Asegurar que todas las características tengan la misma longitud
    all_features = np.concatenate((mfcc_features, chroma.mean(axis=1)[:max_length], contrast.mean(axis=1)[:max_length], tonnetz.mean(axis=1)[:max_length], tempogram[:max_length]))
    # Rellenar con ceros si es necesario
    if len(all_features) < max_length:
        all_features = np.pad(all_features, (0, max_length - len(all_features)))
    return all_features

In [2]:
import os

def listar_archivos_carpeta(ruta_carpeta):
    archivos = [arch for arch in os.listdir(ruta_carpeta) if arch.endswith(('.mp3'))]
    return archivos

In [4]:
ruta_carpeta = "pt"
archivos_carpeta = listar_archivos_carpeta(ruta_carpeta)

# Almacena las características y las etiquetas (nombre de archivo) en listas
caracteristicas = []
etiquetas = []

for archivo in archivos_carpeta:
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    features = extract_features(ruta_archivo)
    caracteristicas.append(features)
    etiquetas.append(archivo)

ModuleNotFoundError: No module named 'numba'

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
caracteristicas_normalizadas = scaler.fit_transform(caracteristicas)

In [41]:
from sklearn.neighbors import NearestNeighbors

# Definir el modelo k-NN
knn_modelo = NearestNeighbors(n_neighbors=5, metric='euclidean')  # Ajusta el número de vecinos según tus necesidades
# Ajustar el modelo a las características normalizadas
knn_modelo.fit(caracteristicas_normalizadas)



NearestNeighbors(metric='euclidean')

In [42]:
# Encontrar vecinos más cercanos para la nueva canción

nueva_cancion_ruta = "pt/Don Omar, Lucenzo - Danza Kuduro.mp3"

# Extraer características de la nueva canción
nueva_cancion_features = extract_features(nueva_cancion_ruta)

# Normalizar las características de la nueva canción (si es necesario)
nueva_cancion_features = scaler.transform([nueva_cancion_features])

# Encontrar vecinos más cercanos para la nueva canción
nueva_cancion_vecinos = knn_modelo.kneighbors(nueva_cancion_features)[1]

# Imprimir los nombres de las canciones más similares
print("Canciones más similares:")
for vecino_idx in nueva_cancion_vecinos[0]:
    print(etiquetas[vecino_idx])

Canciones más similares:
Don Omar, Lucenzo - Danza Kuduro.mp3
NGC Daddy - Love And Choppa.mp3
Pabllo Vittar - Amor de Que.mp3
Majur, Hiran - Naufrago.mp3
Anitta, Lexa, Luísa Sonza - Combatchy (feat. MC Rebecca).mp3


## SIN INDICES:

In [43]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from queue import PriorityQueue


In [44]:
caracteristicas = np.array(caracteristicas)
nueva_cancion_features = np.array(nueva_cancion_features)

In [45]:
def knn_search_priority_queue(query, k):
    distances = euclidean_distances(query.reshape(1, -1), caracteristicas)[0]
    priority_queue = PriorityQueue()
    for i, dist in enumerate(distances):
        priority_queue.put((dist, etiquetas[i]))
    neighbors = []
    for _ in range(k):
        dist, neighbor = priority_queue.get()
        neighbors.append((neighbor, dist))

    return neighbors

In [46]:
def range_search(query, radius):
    distances = euclidean_distances(query.reshape(1, -1), caracteristicas)[0]
    result = [(etiquetas[i], dist) for i, dist in enumerate(distances) if dist <= radius]
    return result

In [53]:
k = 5
radius_values = [70000]

# Búsqueda KNN con cola de prioridad
knn_result_priority_queue = knn_search_priority_queue(nueva_cancion_features, k)
print(f"Vecinos más cercanos (KNN con cola de prioridad): {knn_result_priority_queue}")

# Búsqueda por rango
for radius in radius_values:
    range_result = range_search(nueva_cancion_features, radius)
    print(f"Resultados para rango {radius}: {range_result}")

Vecinos más cercanos (KNN con cola de prioridad): [('NGC Daddy - Love And Choppa.mp3', 60370.67716677054), ('Pabllo Vittar - Amor de Que.mp3', 73399.25259863255), ('Anitta, Lexa, Luísa Sonza - Combatchy (feat. MC Rebecca).mp3', 73936.34466125684), ('Don Omar, Lucenzo - Danza Kuduro.mp3', 85502.71466146605), ('Majur, Hiran - Naufrago.mp3', 86478.74804816848)]
Resultados para rango 70000: [('NGC Daddy - Love And Choppa.mp3', 60370.67716677054)]


## CON INDICES

In [55]:
from rtree import index

^C


## FAISS

In [63]:
import faiss

ImportError: DLL load failed while importing _swigfaiss: No se puede encontrar el módulo especificado.